## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-15-52-16 +0000,nypost,Nearly 250K people displaced by deadly floods ...,https://nypost.com/2025/08/28/world-news/nearl...
1,2025-08-28-15-51-32 +0000,bbc,"UK, France and Germany move to reimpose UN san...",https://www.bbc.com/news/articles/c4gmr3z49lxo...
2,2025-08-28-15-48-57 +0000,nyt,CDC Standoff: Kennedy’s Push to Fire Director ...,https://www.nytimes.com/2025/08/28/health/rfk-...
3,2025-08-28-15-47-00 +0000,wsj,A look at the policymakers who get an equal sa...,https://www.wsj.com/economy/central-banking/fe...
4,2025-08-28-15-44-01 +0000,nyt,European Nations Move to Restart Iran Sanction...,https://www.nytimes.com/2025/08/28/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,53
54,minneapolis,19
56,school,19
57,shooting,16
293,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
191,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,141
47,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,120
5,2025-08-28-15-43-00 +0000,wsj,Federal Reserve governor Lisa Cook sued Presid...,https://www.wsj.com/economy/central-banking/li...,118
18,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...,114
23,2025-08-28-15-14-52 +0000,bbc,Can Trump fire Fed Governor Lisa Cook and why ...,https://www.bbc.com/news/articles/cedvj2d5538o...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
191,141,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
78,83,2025-08-28-11-59-49 +0000,nypost,Minneapolis school shooter Robin Westman confe...,https://nypost.com/2025/08/28/us-news/minneapo...
169,76,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
18,61,2025-08-28-15-24-39 +0000,bbc,US Fed Governor Lisa Cook sues Trump over his ...,https://www.bbc.com/news/articles/c1dxl6ry4y3o...
65,43,2025-08-28-13-03-00 +0000,bbc,Minneapolis mourns two children killed in shoo...,https://www.bbc.com/news/articles/cy40znde9eko...
165,43,2025-08-28-04-19-34 +0000,nypost,Jared Kushner and Tony Blair join Trump for Ga...,https://nypost.com/2025/08/28/us-news/jared-ku...
76,39,2025-08-28-12-02-19 +0000,nypost,Jury awards $2M to movie director shot in face...,https://nypost.com/2025/08/28/us-news/jury-awa...
260,36,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
194,36,2025-08-28-00-45-00 +0000,wsj,Tulsi Gabbard surprised CIA officials last wee...,https://www.wsj.com/politics/national-security...
67,34,2025-08-28-13-01-00 +0000,wsj,Denmark Summons U.S. Envoy Over Suspected Amer...,https://www.wsj.com/world/suspected-american-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
